In [9]:
import pandas as pd

In [22]:
#Loading all decoded CSVs

#The process will be to merge first home_basic and home_data on the Household ID column and person_basic and person data on the
#Person ID column. After that a new Household ID column will be created in the merged person dataframe from the Person ID column
#since the second is an extension of the first). Then I will merge the two remaining dataframes into one with all the person and
#household variables.

try:
    home_basic = pd.read_csv('home_basic_decod.csv', low_memory=False)
    home_data = pd.read_csv('home_data_decod.csv', low_memory=False)
    person_basic = pd.read_csv('person_basic_decod.csv', low_memory=False)
    person_data = pd.read_csv('person_data_decod.csv', low_memory=False)

except FileNotFoundError:
    url1 = 'https://drive.google.com/file/d/1w6LsT3V1us3OGkX6HBSm-0Ou1MEiNIVC/view?usp=sharing'
    path1 = 'https://drive.google.com/uc?id=' + url1.split('/')[-2]
    home_basic = pd.read_csv(path1, low_memory=False)
    
    url2 = 'https://drive.google.com/file/d/1RuZZPNeROvK-gGvt2XbbCd_KfXf6q-5_/view?usp=sharing'
    path2 = 'https://drive.google.com/uc?id=' + url2.split('/')[-2]
    home_data = pd.read_csv(path2, low_memory=False)
    
    url3 = 'https://drive.google.com/file/d/1oUvTyw29slQPsav-F1fT0tjXsdBPkUaW/view?usp=sharing'
    path3 = 'https://drive.google.com/uc?id=' + url3.split('/')[-2]
    person_basic = pd.read_csv(path3, low_memory=False)
    
    url4 = 'https://drive.google.com/file/d/1uZIJpAtkexhapcaWr9Adui5B2bX8j4Ql/view?usp=sharing'
    path4 = 'https://drive.google.com/uc?id=' + url4.split('/')[-2]
    person_data = pd.read_csv(path4, low_memory=False)

In [23]:
#Removing old index columns

home_basic = home_basic.drop(['Unnamed: 0'], axis=1)
home_data = home_data.drop(['Unnamed: 0'], axis=1)
person_basic = person_basic.drop(['Unnamed: 0'], axis=1)
person_data = person_data.drop(['Unnamed: 0'], axis=1)

In [24]:
#This function finds the names of all common columns between two dataframes. This way I can remove them before merging.

def duplicated_columns(df1, df2):
    duplicated_columns = []
    for c1 in df1.columns:
        for c2 in df2.columns:
            if c1 == c2:
                duplicated_columns.append(c1)
    return duplicated_columns

duplicated_home = duplicated_columns(home_basic, home_data)
duplicated_person = duplicated_columns(person_basic, person_data)

print(duplicated_home)
print(duplicated_person)

['Year of survey', 'Country', 'Household ID']
['Year of survey', 'Country', 'Personal ID', 'Personal cross-sectional weight', 'Personal cross-sectional weight (flag)', 'Month of birth', 'Month of birth (flag)', 'Year of birth', 'Year of birth (flag)', 'Sex', 'Sex (flag)', 'Father ID', 'Father ID (flag)', 'Mother ID', 'Mother ID (flag)', 'Spouse/partner ID', 'Spouse/partner ID (flag)']


In [25]:
#I intend to use the ID columns as merge keys so I substract them from my duplicated columns lists.

duplicated_home.remove('Household ID')
duplicated_person.remove('Personal ID')

In [26]:
#Getting rid of duplicated columns

home_data = home_data.drop(duplicated_home, axis=1)
person_data = person_data.drop(duplicated_person, axis=1)

In [27]:
#Merging dataframes

merged_home = pd.merge(home_basic, home_data, on=['Household ID'], how='left')
merged_person = pd.merge(person_basic, person_data, on=['Personal ID'], how='left')

In [28]:
#All personal IDs are made up of the household ID plus a number from 0-99. This function adds a household ID column to the 
#person dataframe to use as key for the last merge.

def household_id(integer):
    return int(str(integer)[:-2])

merged_person['Household ID'] = merged_person['Personal ID'].apply(household_id)

In [29]:
#Getting duplicates

duplicated_data = duplicated_columns(merged_person, merged_home)

print(duplicated_data)

['Year of survey', 'Country', 'Household ID']


In [30]:
#Except ID

duplicated_data.remove('Household ID')

In [31]:
#Removing duplicates

merged_home = merged_home.drop(duplicated_data, axis=1)

In [32]:
#Merging

merged_data = pd.merge(merged_person, merged_home, on=['Household ID'], how='left')

In [27]:
merged_data.to_csv('ilc.csv')